In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
import operator

In [2]:
target = 'redemption_status'

features = ['campaign_type', 'duration', 'marital_status', 'rented', 'no_of_children', 'income_bracket', 'c_brand_count', 'c_brand_freq', 'c_brand_rare', 'c_brand_freq_count', 'c_brand_rare_count', 'c_category_freq_count', 'c_category_rare_count', 't_brand_count', 't_brand_rare', 't_brand_freq_count', 't_category_count', 't_category_rare', 't_category_rare_count', 'min_selling_price', 'max_selling_price', 'mean_selling_price', 'min_other_discount', 'mean_other_discount', 'min_coupon_discount', 'max_coupon_discount', 'mean_coupon_discount', 'min_total_discount', 'max_total_discount', 'mean_total_discount', 'min_selling_price_pq', 'max_selling_price_pq', 'mean_selling_price_pq', 'min_other_discount_pq', 'max_other_discount_pq', 'mean_other_discount_pq', 'min_total_discount_pq', 'mean_total_discount_pq', 'transc_coupon', 'transc_other', 'transc_any', 'quantity', 'c_transc_other', 'c_transc_any', 'c_quantity', 'brand_freq']

categorical_columns = ['campaign_type', 'age_range', 'marital_status', 'rented', 'family_size', 'no_of_children', 'income_bracket', 'c_brand_freq', 'c_brand_rare', 'c_brand_type_freq', 'c_brand_type_rare', 'c_category_freq', 'c_category_rare', 't_brand_freq', 't_brand_rare', 't_brand_type_freq', 't_brand_type_rare', 't_category_freq', 't_category_rare', 'brand_freq', 'brand_rare', 'brand_type_freq', 'brand_type_rare', 'category_freq', 'category_rare']

merge_columns = ['brand_freq', 'brand_rare', 'brand_type_freq', 'brand_type_rare', 'category_freq', 'category_rare']

In [3]:
def preprocess(trainset, testset):        
    dataset = trainset.append(testset, sort=False).fillna(0)
    
    for column in merge_columns:
        dataset[column] = (dataset['t_'+column] == dataset['c_'+column]).astype('int')
        
    for column in categorical_columns:
        dataset[column] = dataset[column].astype('category')
    
    dataset = dataset[features] 
#     dataset = pd.get_dummies(dataset[features])
#     dataset = dataset.drop(encoded_columns, axis=1)
    
    trainset = trainset[[target]].join(dataset)
    testset = testset[[]].join(dataset)
    
    return trainset, testset

In [4]:
trainset = pd.read_csv('data/train/train_feature.csv', index_col='id', parse_dates=['start_date','end_date'])
trainset.head()

,redemption_status,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,age_range,marital_status,...,max_total_discount_pq,mean_total_discount_pq,transc_coupon,transc_other,transc_any,quantity,c_transc_coupon,c_transc_other,c_transc_any,c_quantity
id,,,,,,,,,,,,,,,,,,,,,
1,0,13,27,1053,0,2013-05-19,2013-07-05,47,46-55,Single,...,NaN,NaN,0.000000,0.561798,0.561798,29273,0.0,0.0,0.0,0
2,0,13,116,48,0,2013-05-19,2013-07-05,47,36-45,Married,...,NaN,NaN,0.028986,0.507246,0.536232,75,0.0,0.0,0.0,0
6,0,9,635,205,1,2013-03-11,2013-04-12,32,46-55,Married,...,NaN,NaN,0.081481,0.503704,0.529630,372,0.0,0.0,0.0,0
7,0,13,644,1050,0,2013-05-19,2013-07-05,47,NaN,NaN,...,NaN,NaN,0.000000,0.524590,0.524590,69,0.0,0.0,0.0,0
9,0,8,1017,1489,0,2013-02-16,2013-04-05,48,46-55,Married,...,NaN,NaN,0.000000,0.514085,0.514085,37064,0.0,0.0,0.0,0


In [5]:
testset = pd.read_csv('data/test/test_feature.csv', index_col='id', parse_dates=['start_date','end_date'])
testset.head()

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,age_range,marital_status,rented,...,max_total_discount_pq,mean_total_discount_pq,transc_coupon,transc_other,transc_any,quantity,c_transc_coupon,c_transc_other,c_transc_any,c_quantity
id,,,,,,,,,,,,,,,,,,,,,
3,22,869,967,0,2013-09-16,2013-10-18,32,36-45,Single,0.0,...,0.0,-17.81,0.032028,0.473310,0.483986,15405,0.0,0.5,0.5,2
4,20,389,1566,1,2013-09-07,2013-11-16,70,26-35,Married,0.0,...,0.0,-17.63,0.004967,0.458609,0.461921,65687,0.0,0.5,0.5,2
5,22,981,510,0,2013-09-16,2013-10-18,32,26-35,Single,0.0,...,NaN,NaN,0.007792,0.490909,0.493506,31089,0.0,0.0,0.0,0
8,25,1069,361,1,2013-10-21,2013-11-22,32,18-25,Single,0.0,...,NaN,NaN,0.000000,0.517241,0.517241,21785,0.0,0.0,0.0,0
10,17,498,811,1,2013-07-29,2013-08-30,32,NaN,NaN,NaN,...,NaN,NaN,0.002288,0.473684,0.473684,544,0.0,0.0,0.0,0


In [6]:
trainset, testset = preprocess(trainset, testset)

print("Trainset size: {}".format(trainset.shape))
print("Testset size: {}".format(testset.shape))

Trainset size: (78369, 47)
Testset size: (50226, 46)


In [7]:
feature_columns = features
trainset, validationset = train_test_split(trainset, random_state=41, test_size=0.2)

In [8]:
def train(features):
    X_train, y_train = trainset[features], trainset[target]
    X_val, y_val = validationset[features], validationset[target]
    
    lgb = LGBMClassifier(random_state=41, n_jobs=4)
    lgb.fit(X_train, y_train, eval_metric='auc')
    y_pred = lgb.predict_proba(X_val)[:,1]
    
    score = roc_auc_score(y_val, y_pred)
    return score, lgb

def exclude_feature(features, index):
    new_features = features[:index] + features[index+1:]
    score, model = train(new_features)
    return score

In [9]:
all_score, all_model = train(feature_columns)
all_score

0.92365117963017

Feature Verification

In [10]:
score_map = {name : (exclude_feature(feature_columns, index) - all_score) 
             for index, name in enumerate(feature_columns)}
score_map = sorted(score_map.items(), key=operator.itemgetter(1), reverse=True)
score_map

[('max_selling_price', -0.001832074268607542),
 ('min_other_discount', -0.002786153530244473),
 ('mean_other_discount', -0.0037389359011769097),
 ('c_brand_rare_count', -0.003870786456138031),
 ('t_category_rare_count', -0.003899318051637901),
 ('marital_status', -0.004444012147542975),
 ('min_total_discount_pq', -0.005505301040754751),
 ('c_brand_freq', -0.005661792519102038),
 ('min_selling_price', -0.005807044278010154),
 ('quantity', -0.005908634049865302),
 ('min_selling_price_pq', -0.006276518713052126),
 ('no_of_children', -0.006801759448389122),
 ('c_category_freq_count', -0.006999751429281553),
 ('duration', -0.007432048330793584),
 ('income_bracket', -0.007701801597337021),
 ('min_total_discount', -0.007882933999070585),
 ('max_coupon_discount', -0.00797587783289555),
 ('rented', -0.007990575927547039),
 ('t_category_rare', -0.008109025278561388),
 ('min_coupon_discount', -0.008165223875757799),
 ('t_category_count', -0.008229203817181618),
 ('mean_other_discount_pq', -0.0083

In [11]:
X_test = testset[feature_columns]
testset[target] = all_model.predict_proba(X_test)[:,1]

submission = testset[[target]]
submission.to_csv('data/report/report_1.csv')